In [1]:
!pip install transformers
!pip install tensorboardx
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 4.6 MB/s eta 0:00:0000:01


# Necessary Imports

In [2]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
import os
from PIL import Image
from scipy.fftpack import fft
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
import csv
# from file_util import *

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# File util

In [3]:
#-*- coding: utf-8 -*-


import os
import chardet

'''
dirname        : path that need to be searched
ret                : files in the dirname (recursive)
list_avoid_dir : dirname need to be skipped
usage           : 
    list_files = []
    file_search(dirname, list_files):   
'''
def file_search(dirname, ret, list_avoid_dir=[]):
    
    filenames = os.listdir(dirname)
    
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)

        if os.path.isdir(full_filename) :
            if full_filename.split('/')[-1] in list_avoid_dir:
                continue
            else:
                file_search(full_filename, ret, list_avoid_dir)
            
        else:
            ret.append( full_filename )          

            

'''
filename : filename (inc. path) that will be inspected
'''
def find_encoding(filename):
    rawdata = open(filename, 'rb').read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']    
    return charenc
            
'''
dir_name : dir_name (inc. path) that will be created ( full-path name )
'''
def create_folder(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

# Extract Label Transcription

In [4]:
out_file = '/kaggle/working/processed_tran.csv'
os.system('rm ' + out_file)
def extract_trans( list_in_file, out_file ) :
    lines = []
    for in_file in list_in_file:
        cnt = 0
        
        with open(in_file, 'r') as f:
            lines = f.readlines()

        with open(out_file, 'a') as f:

            csv_writer = csv.writer( f )
            lines = sorted(lines)                  # sort based on first element
            
            for line in lines:

                name = line.split(':')[0].split(' ')[0].strip()
                
                # unwanted case 
                if name[:3] != 'Ses':             # noise transcription such as reply  M: sorry
                    continue
                elif name[-3:-1] == 'XX':        # we don't have matching pair in label
                    continue
                trans = line.split(':')[1].strip()
                
                cnt += 1
                csv_writer.writerow([name, trans])

list_files = []

for x in range(5):
    sess_name = 'Session' + str(x+1)

    path = '/kaggle/input/iemocapfullrelease/IEMOCAP_full_release/' + sess_name + '/dialog/transcriptions/'
    file_search(path, list_files)
    list_files = sorted(list_files)

    print(sess_name + ", #sum files: " + str(len(list_files)))

extract_trans(list_files, out_file)

rm: cannot remove '/kaggle/working/processed_tran.csv': No such file or directory


Session1, #sum files: 28
Session2, #sum files: 58
Session3, #sum files: 90
Session4, #sum files: 120
Session5, #sum files: 151


In [5]:
out_file = '/kaggle/working/label.csv'
list_category = [
                'ang',
                'hap',
                'sad',
                'neu',
                'fru',
                'exc',
                'fea',
                'sur',
                'dis',
                'oth',
                'xxx'
                ]

category = {}
for c_type in list_category:
    if c_type in category:
        ;
    else:
        category[c_type] = len(category)

In [6]:
def find_category(lines):
    is_target = True
    
    id = ''
    c_label = ''
    list_ret = []
    
    for line in lines:
        
        if is_target == True:
            
            try:
                id = line.split('\t')[1].strip()  #  extract ID
                c_label  = line.split('\t')[2].strip()  #  extract category
                if c_label not in category:
                    print("ERROR nokey" + c_label)
                    sys.exit()
                
                list_ret.append( [id, c_label] )
                is_target = False

            except:
                print("ERROR " + line)
                sys.exit()
        
        else:
            if line == '\n':
                is_target = True
    return list_ret

def extract_labels( list_in_file, out_file ) :
    id = ''
    lines = []
    list_ret = []
    
    for in_file in list_in_file:
        
        with open(in_file, 'r') as f:
            lines = f.readlines()
            lines = lines[2:]                           # remove head
            list_ret = find_category(lines)
            
        list_ret = sorted(list_ret)                   # sort based on first element
    
        with open(out_file, 'a') as f:
            csv_writer = csv.writer( f )
            csv_writer.writerows( list_ret )  

In [7]:
list_files = []
list_avoid_dir = ['Attribute', 'Categorical', 'Self-evaluation']

for x in range(5):
    sess_name = 'Session' + str(x+1)

    path = '/kaggle/input/iemocapfullrelease/IEMOCAP_full_release/' + sess_name + '/dialog/EmoEvaluation/'
    file_search(path, list_files, list_avoid_dir)
    list_files = sorted(list_files)

    print(sess_name + ", #sum files: " + str(len(list_files)))

extract_labels(list_files, out_file)

Session1, #sum files: 28
Session2, #sum files: 58
Session3, #sum files: 90
Session4, #sum files: 120
Session5, #sum files: 151


In [8]:
lines = []
with open('/kaggle/working/label.csv') as f :
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]

In [9]:
df = pd.DataFrame(columns=['sessionID', 'label'])
with open('/kaggle/working/processed_label.txt', 'w') as f:
    
    with open('/kaggle/working/processed_ids.txt', 'w') as f2:
    
        for line in lines:
            if line[1] == 'ang':
                f.write('0\n')
                f2.write(line[0]+'\n')
                df.loc[len(df.index)] = [line[0], 0] 
            elif line[1] == 'hap':
                f.write('1\n')
                f2.write(line[0]+'\n')
                df.loc[len(df.index)] = [line[0], 1] 
            elif line[1] == 'exc':
                f.write('1\n')
                f2.write(line[0]+'\n')
                df.loc[len(df.index)] = [line[0], 1] 
            elif line[1] == 'sad':
                f.write('2\n')
                f2.write(line[0]+'\n')
                df.loc[len(df.index)] = [line[0], 2] 
            elif line[1] == 'neu':
                f.write('3\n')
                f2.write(line[0]+'\n')
                df.loc[len(df.index)] = [line[0], 3] 
            else :
                f.write('-1\n')
                df.loc[len(df.index)] = [line[0], -1] 

In [10]:
lines = []
with open('/kaggle/working/processed_label.txt') as f :
    lines = f.readlines()
lines = [x.strip() for x in lines]

print(len([x for x in lines if x=='0']))
print(len([x for x in lines if x=='1']))
print(len([x for x in lines if x=='2']))
print(len([x for x in lines if x=='3']))

1103
1636
1084
1708


In [11]:
# Verify

in_file = '/kaggle/working/processed_label.txt'
label = []
with open( in_file, 'r') as f:
    label = f.readlines()
    
label_id = [ x.strip() for x in label ]

in_file = '/kaggle/working/label.csv'
label = []
with open( in_file, 'r') as f:
    csv_reader = csv.reader( f )
    label = [x for x in csv_reader]
    
label_id = [ x[0] for x in label]

in_file = '/kaggle/working/processed_tran.csv'
tran = []
with open( in_file, 'r') as f:
    csv_reader = csv.reader( f )
    tran = [x for x in csv_reader]
    
tran_id = [ x[0] for x in tran]

for l, t in zip(label_id, tran_id):
    if l != t:
        print('ERROR')

# Paper: Attention Based Fully Convolutional Network for Speech Emotion Recognition

In [12]:
def audio2spectrogram(filepath):
    #fig = plt.figure(figsize=(5,5))
    samplerate, test_sound  = wavfile.read(filepath,mmap=True)
    #print('samplerate',samplerate)
    _, spectrogram = log_specgram(test_sound, samplerate)
    #print(spectrogram.shape)
    #print(type(spectrogram))
    #plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    return spectrogram
    
def audio2wave(filepath):
    fig = plt.figure(figsize=(5,5))
    samplerate, test_sound  = wavfile.read(filepath,mmap=True)
    plt.plot(test_sound)
    
def log_specgram(audio, sample_rate, window_size=40,
                 step_size=20, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    #print('noverlap',noverlap)
    #print('nperseg',nperseg)
    freqs, _, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, np.log(spec.T.astype(np.float32) + eps)

N_CHANNELS = 3
def get_3d_spec(Sxx_in, moments=None):
    if moments is not None:
        (base_mean, base_std, delta_mean, delta_std,
             delta2_mean, delta2_std) = moments
    else:
        base_mean, delta_mean, delta2_mean = (0, 0, 0)
        base_std, delta_std, delta2_std = (1, 1, 1)
    h, w = Sxx_in.shape
    right1 = np.concatenate([Sxx_in[:, 0].reshape((h, -1)), Sxx_in], axis=1)[:, :-1]
    delta = (Sxx_in - right1)[:, 1:]
    delta_pad = delta[:, 0].reshape((h, -1))
    delta = np.concatenate([delta_pad, delta], axis=1)
    right2 = np.concatenate([delta[:, 0].reshape((h, -1)), delta], axis=1)[:, :-1]
    delta2 = (delta - right2)[:, 1:]
    delta2_pad = delta2[:, 0].reshape((h, -1))
    delta2 = np.concatenate([delta2_pad, delta2], axis=1)
    base = (Sxx_in - base_mean) / base_std
    delta = (delta - delta_mean) / delta_std
    delta2 = (delta2 - delta2_mean) / delta2_std
    stacked = [arr.reshape((h, w, 1)) for arr in (base, delta, delta2)]
    return np.concatenate(stacked, axis=2)

# Extract the MFCC feature

In [13]:
list_files = []
for x in range(5):
    sess_name = 'Session' + str(x+1)
    path = '/kaggle/input/iemocapfullrelease/IEMOCAP_full_release/'+ sess_name + '/sentences/wav/'
    file_search(path, list_files)
    list_files = sorted(list_files)
    print (sess_name + ", #sum files: " + str(len(list_files)))

Session1, #sum files: 1820
Session2, #sum files: 3633
Session3, #sum files: 5769
Session4, #sum files: 7873
Session5, #sum files: 10043


In [14]:
df.head()

,sessionID,label
0,Ses01F_impro01_F000,3
1,Ses01F_impro01_F001,3
2,Ses01F_impro01_F002,3
3,Ses01F_impro01_F003,-1
4,Ses01F_impro01_F004,-1


In [15]:
no_rows=len(list_files)
index=0
sprectrogram_shape=[]
docs = []
bookmark=0
extraLabel=0
for everyFile in list_files:
  if(everyFile.split('/')[-1].endswith('.wav')):
    filename=everyFile.split('/')[-1].strip('.wav')
    lable=df.loc[df['sessionID']==filename]['label'].values[0]
#     print('label',lable)
    if(lable!=-1):
      #sprectrogram_shape.append(audio2spectrogram(everyFile))
      spector=audio2spectrogram(everyFile)
      spector=get_3d_spec(spector)
      npimg = np.transpose(spector,(2,0,1))
      input_tensor=torch.tensor(npimg)
      input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
      #X, sample_rate = librosa.load(everyFile, res_type='kaiser_fast',sr=22050*2)
      #sample_rate = np.array(sample_rate)
      #mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=13),axis=0)
      #feature = mfccs
      docs.append({
         'fileName':everyFile.split('/')[-1].strip('.wav'),
         #'feature_mfcc':feature,
         'sprectrome':input_batch,
         'label':lable
              })
      index+=1
#       print('index',index)
    else:
      extraLabel=extraLabel+1
#       print('extraLabel',extraLabel)

# TestAlexnet input

In [16]:
import torch
import torch.nn as nn
#from .utils import load_state_dict_from_url
from torch.hub import load_state_dict_from_url

__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.num_classes=num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((12, 12))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        print('features',x.shape)
        
        #x = self.avgpool(x)
        #print('avgpool',x.shape)
        #x = torch.flatten(x, 1)
        #print('flatten',x.shape)
        #x = self.classifier(x)
        return x
def alexnet(pretrained=False, progress=True, **kwargs):
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

# Modified Alexnet

In [17]:
class ModifiedAlexNet(nn.Module):
    def __init__(self, num_classes=4):
        super(ModifiedAlexNet, self).__init__()
        self.num_classes=num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.features(x)
        #print('features',x.shape)
        x=torch.flatten(x, start_dim=2)#a1,a2,a3......al{a of dim c} 
        x=torch.sum(x, dim=2)#a1*alpha1+a2*alpha2+.......+al*alphal
        #print(x.shape)
        x=self.classifier(x)
        #print('classifier',x)
        #x=self.softmax(x)
        #print('softmax',x)
        #x = self.avgpool(x)
        #print('avgpool',x.shape)
        #x = torch.flatten(x, 1)
        #print('flatten',x.shape)
        #x = self.classifier(x)
        return x
   
def modifiedAlexNet(pretrained=False, progress=True, **kwargs):
    model_modified = ModifiedAlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model_modified.load_state_dict(state_dict)
    return model_modified

In [18]:
original_model=alexnet(pretrained=True)
original_dict = original_model.state_dict()
modifiedAlexNet=modifiedAlexNet(pretrained=False)
modified_model_dict = modifiedAlexNet.state_dict()
pretrained_modified_model_dict = {k: v for k, v in original_dict.items() if k in modified_model_dict}
modifiedAlexNet.to('cuda')

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth
100%|██████████| 233M/233M [00:01<00:00, 155MB/s]  


ModifiedAlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=256, out_features=4, bias=True)
  )
  (softmax): Softmax(dim=1)
)

In [19]:

x=audio2spectrogram(list_files[40])
x=get_3d_spec(x)
npimg = np.transpose(x,(2,0,1))
input_tensor=torch.tensor(npimg)

input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    modifiedAlexNet.to('cuda')
with torch.no_grad():
    output = modifiedAlexNet(input_batch)
    #output.squeeze().shape
    #output=torch.flatten(output, start_dim=2)
    #print(output.shape)
    #output=torch.sum(output, dim=2)
    print(output)

tensor([[-5.0555, -9.2828,  2.3354, -2.1421]], device='cuda:0')


In [20]:
import random
random.shuffle(docs)
random.shuffle(docs)
random.shuffle(docs)
total_length=len(docs)
train_length=int(.9*total_length)
train_list=docs[0:train_length]
test_list=docs[train_length:]
print('no of items for train ',len(train_list))
print('no of items for test ',len(test_list))

no of items for train  4977
no of items for test  554


In [21]:
for name, param in modifiedAlexNet.named_parameters():
      if(param.requires_grad):
        print(name)
      else:
        print('no grad',name)

features.0.weight
features.0.bias
features.3.weight
features.3.bias
features.6.weight
features.6.bias
features.8.weight
features.8.bias
features.10.weight
features.10.bias
classifier.1.weight
classifier.1.bias


In [22]:
import torch.optim as optim
from transformers import AdamW
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(modifiedAlexNet.parameters(),
                  lr =  2e-4, 
                  eps = 1e-8
                )
from transformers import get_linear_schedule_with_warmup

NUM_EPOCHS=16

writer = SummaryWriter(log_dir='/kaggle/working/content/')
total_steps = len(train_list) * NUM_EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Training loop

In [23]:
total_steps = 1


seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch in range(NUM_EPOCHS):
  modifiedAlexNet.train()
  for every_trainlist in train_list:
    label1=every_trainlist['label']
    label1=torch.tensor([label1])
    sprectrome=every_trainlist['sprectrome']
    if(sprectrome.shape[2]>65):
      optimizer.zero_grad()
      sprectrome = sprectrome.to('cuda')
      label1=label1.to('cuda')
      modifiedAlexNet.zero_grad()
      output = modifiedAlexNet(sprectrome)
      #print('softmax output ',output)
      loss = criterion(output, label1)
      #print('label1',label1)
      #print('loss',loss.item())
      loss.backward()
      torch.nn.utils.clip_grad_norm_(modifiedAlexNet.parameters(), 1.0)
      optimizer.step()
      scheduler.step()
      _, preds = torch.max(output, 1)
      accuracy = torch.sum(preds == label1)
      #print('accuracy.item()',accuracy.item())
      #print('preds',preds)
      if total_steps % 10 == 0:
        with torch.no_grad():
          _, preds = torch.max(output, 1)
          accuracy = torch.sum(preds == label1)
          print('Epoch: {} \tStep: {} \tLoss: {:.4f} \tAcc: {}'.format(epoch + 1, total_steps, loss.item(), accuracy.item()))
#           tbwriter.add_scalar('loss', loss.item(), total_steps)
#           tbwriter.add_scalar('accuracy', accuracy.item(), total_steps)                     
      total_steps+=1

Epoch: 1 	Step: 10 	Loss: 2.0132 	Acc: 0
Epoch: 1 	Step: 20 	Loss: 1.0164 	Acc: 1
Epoch: 1 	Step: 30 	Loss: 0.8774 	Acc: 1
Epoch: 1 	Step: 40 	Loss: 1.7744 	Acc: 0
Epoch: 1 	Step: 50 	Loss: 1.4711 	Acc: 0
Epoch: 1 	Step: 60 	Loss: 1.5994 	Acc: 0
Epoch: 1 	Step: 70 	Loss: 2.0185 	Acc: 0
Epoch: 1 	Step: 80 	Loss: 1.3879 	Acc: 0
Epoch: 1 	Step: 90 	Loss: 6.8684 	Acc: 0
Epoch: 1 	Step: 100 	Loss: 1.5992 	Acc: 0
Epoch: 1 	Step: 110 	Loss: 0.5372 	Acc: 1
Epoch: 1 	Step: 120 	Loss: 1.4614 	Acc: 0
Epoch: 1 	Step: 130 	Loss: 1.5113 	Acc: 0
Epoch: 1 	Step: 140 	Loss: 0.6491 	Acc: 1
Epoch: 1 	Step: 150 	Loss: 0.9022 	Acc: 0
Epoch: 1 	Step: 160 	Loss: 9.6038 	Acc: 0
Epoch: 1 	Step: 170 	Loss: 0.5775 	Acc: 1
Epoch: 1 	Step: 180 	Loss: 2.5653 	Acc: 0
Epoch: 1 	Step: 190 	Loss: 1.0215 	Acc: 0
Epoch: 1 	Step: 200 	Loss: 1.3154 	Acc: 1
Epoch: 1 	Step: 210 	Loss: 1.2112 	Acc: 1
Epoch: 1 	Step: 220 	Loss: 1.3621 	Acc: 0
Epoch: 1 	Step: 230 	Loss: 1.1729 	Acc: 1
Epoch: 1 	Step: 240 	Loss: 0.9427 	Acc: 1
E

In [24]:
torch.save(modifiedAlexNet, '/kaggle/working//model_audio_new_opt.pt')
model=torch.load('/kaggle/working//model_audio_new_opt.pt')
model.eval()
model.to('cpu')

ModifiedAlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=256, out_features=4, bias=True)
  )
  (softmax): Softmax(dim=1)
)

In [25]:
y_actu=[]
y_pred=[]
for every_test_list in test_list:
    label1=every_test_list['label']
    label1=torch.tensor([label1])
    sprectrome=every_test_list['sprectrome']
    with torch.no_grad():
      if(sprectrome.shape[2]>65):
        #sprectrome = sprectrome.to('cuda')
        #label1=label1.to('cuda')
        output = model(sprectrome)
        _, preds = torch.max(output, 1)
        y_actu.append(label1.numpy()[0])
        y_pred.append(preds.numpy()[0])

In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_actu, y_pred)

array([[61, 26,  6, 20],
       [27, 70,  8, 41],
       [ 0, 17, 74, 20],
       [17, 42, 17, 84]])

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_actu,y_pred))

              precision    recall  f1-score   support

           0       0.58      0.54      0.56       113
           1       0.45      0.48      0.47       146
           2       0.70      0.67      0.69       111
           3       0.51      0.53      0.52       160

    accuracy                           0.55       530
   macro avg       0.56      0.55      0.56       530
weighted avg       0.55      0.55      0.55       530

